# Tarea 3

Alejandro Quiñones - 1463399J

## Detección de Caras

In [153]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from pybalu.performance_eval import performance
from pybalu.feature_selection import sfs, clean
from pybalu.feature_transformation import normalize, pca
from pybalu.data_selection import stratify
from scipy.io import loadmat
import numpy as np

In [154]:
N_FEATURES = 40
N_COMPONENTS = 10

In [155]:
M = loadmat("set05-face-detection.mat")

In [156]:
X = M['f']
Y = M['d']

Se estratifica el dataset con 80-20

In [157]:
clase1_20 = [X[i] for i in range(13)]
clase1_80 = [X[i] for i in range(13, 64)]
clase2_20 = [X[i] for i in range(64, 104)]
clase2_80 = [X[i] for i in range(104, 264)]

y1_20 = [Y[i] for i in range(13)]
y1_80 = [Y[i] for i in range(13, 64)]
y2_20 = [Y[i] for i in range(64, 104)]
y2_80 = [Y[i] for i in range(104, 264)]

In [158]:
Xtrain = clase1_80.copy()
Xtrain.extend(clase2_80)
Xtrain = np.vstack(Xtrain)

Xtest = clase1_20.copy()
Xtest.extend(clase2_20)
Xtest = np.vstack(Xtest)

Ytrain = y1_80.copy()
Ytrain.extend(y2_80)
Ytrain = np.vstack(Ytrain)

Ytest = y1_20.copy()
Ytest.extend(y2_20)
Ytest = np.vstack(Ytest)

Se aplica un clean

In [159]:
p_clean = clean(Xtrain)

In [160]:
Xtrain_clean = Xtrain[:, p_clean]
Xtest_clean = Xtest[:, p_clean]

Se normaliza el dataset

In [161]:
normalizer = Normalizer().fit(Xtrain_clean)
Xtrain_norm = normalizer.transform(Xtrain_clean)
Xtest_norm = normalizer.transform(Xtest_clean)

In [162]:
##Xtrain_norm, a, b = normalize(Xtrain_clean) 
#Xtest_norm = Xtest_clean * a + b

Se seleccionan features según SFS

In [163]:
p_sfs = sfs(Xtrain_norm, Ytrain, n_features=N_FEATURES, show=True)

Selecting Features: 100%|██████████| 40.0/40.0 [00:06<00:00, 5.75 features/s]


In [164]:
Xtrain_sfs = Xtrain_norm[:, p_sfs]
Xtest_sfs = Xtest_norm[:, p_sfs]

Se transforman las features a través de PCA

In [165]:
sk_pca = PCA(n_components=N_COMPONENTS)
Xtrain_pca = sk_pca.fit_transform(Xtrain_sfs)
Xtest_pca = sk_pca.transform(Xtest_sfs)

Se iniciliaza el modelo y se obtiene  la predicción del test set

In [166]:
# Inicializamos el clasificador con k=1
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(Xtrain_pca, Ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [167]:
pred = knn.predict(Xtest_pca)
print(f"Predicción tuvo un accuracy de {performance(pred, Ytest)}")

Predicción tuvo un accuracy de 0.9245283018867925
